<p id = 'toc'></p>

<h1 style = "font-family:Verdana; font-size:24px; font-weight: bold; letter-spacing: 3px; color: teal;"> TABLE OF CONTENTS </h1>

---

[<h3 style = "font-family: Verdana; font-size: 18px; text-indent: 8vw; letter-spacing: 2px; color: teal">0&nbsp;&nbsp;&nbsp;&nbsp; IMPORTS </h3>](#0)

---

<h3 style = "font-family:Verdana; font-size: 18px; text-indent: 8vw; letter-spacing: 2px; color: teal"><a href='#1'>1.&nbsp;&nbsp;&nbsp;&nbsp;DATASET</a></h3>

---

<h3 style = "font-family:Verdana; font-size: 18px; text-indent: 8vw; letter-spacing: 2px; color: teal"><a href='#2'>2.&nbsp;&nbsp;&nbsp;&nbsp;CONVERTING IMAGE DATA TO JPG</a></h3>


<a id = '0'></a>
# <span><center style = "font-family: Verdana; font-size:36px; color : teal; letter-spacing: 3px; font-weight: bold">0. &nbsp;IMPORTS&nbsp;&nbsp;&nbsp;&nbsp;<a href = '#toc'>&#10514;</a></center></span>

In [1]:
# pip install pydicom

In [2]:
import pandas as pd 
import numpy as np
import os
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import torch
import cv2
import torchvision.transforms as transforms
import shutil


from PIL import Image
from tqdm import tqdm
from torchvision import datasets
from pydicom import dcmread

import warnings
warnings.filterwarnings("ignore")

%config Completer.use_jedi = False

In [3]:
class config:
    TRAIN_PATH = '../input/unifesp-x-ray-body-part-classifier/train'
    TRAIN_CSV = '../input/unifesp-x-ray-body-part-classifier/train.csv'
    WORKING_PATH = './'
    TRAIN_JPG_PATH = '../input/unifesp-xray-jpg-dataset/train'
    RANDOM_SEED = 15

<a id="1"></a>

# <span><center style= "font-family:Verdana; font-size:36px; font-weight: bold; letter-spacing: 3px; color: teal">1.&nbsp;&nbsp;DATASET&nbsp;&nbsp;&nbsp;&nbsp;<a href = '#toc'>&#10514;</a> </center></span>

In [4]:
train_df = pd.read_csv(config.TRAIN_CSV)
train_df.head(10)

,SOPInstanceUID,Target
0,1.2.826.0.1.3680043.8.498.10025629581362719970...,0
1,1.2.826.0.1.3680043.8.498.10036150326276641158...,15
2,1.2.826.0.1.3680043.8.498.10038426859954986240...,12
3,1.2.826.0.1.3680043.8.498.10050991192143676483...,14
4,1.2.826.0.1.3680043.8.498.10053309524595490852...,3
5,1.2.826.0.1.3680043.8.498.10053755320637729867...,3
6,1.2.826.0.1.3680043.8.498.10062189329714053601...,14
7,1.2.826.0.1.3680043.8.498.10065930002825553435...,13 20
8,1.2.826.0.1.3680043.8.498.10072001800484199846...,3
9,1.2.826.0.1.3680043.8.498.10077219503857952412...,3


In [5]:
def image_path_creation(path, replace_part):
    file_names = []
    full_paths = []
    for dirpath,  dirname, filenames in os.walk(path):
        if filenames:
            for filename in filenames:
                full_path = os.path.join(dirpath, filename)
                filename = filename.replace(replace_part, '')
                full_paths.append(full_path)
                file_names.append(filename)
    return file_names, full_paths

In [6]:
file_names, full_paths = image_path_creation(config.TRAIN_PATH, '-c.dcm')
image_path_dict = {'SOPInstanceUID': file_names, 'path':full_paths}
image_df = pd.DataFrame(image_path_dict)

In [7]:
train_df = pd.merge(train_df, image_df, on = 'SOPInstanceUID')

In [8]:
train_df.head(8)

,SOPInstanceUID,Target,path
0,1.2.826.0.1.3680043.8.498.10025629581362719970...,0,../input/unifesp-x-ray-body-part-classifier/tr...
1,1.2.826.0.1.3680043.8.498.10036150326276641158...,15,../input/unifesp-x-ray-body-part-classifier/tr...
2,1.2.826.0.1.3680043.8.498.10038426859954986240...,12,../input/unifesp-x-ray-body-part-classifier/tr...
3,1.2.826.0.1.3680043.8.498.10050991192143676483...,14,../input/unifesp-x-ray-body-part-classifier/tr...
4,1.2.826.0.1.3680043.8.498.10053309524595490852...,3,../input/unifesp-x-ray-body-part-classifier/tr...
5,1.2.826.0.1.3680043.8.498.10053755320637729867...,3,../input/unifesp-x-ray-body-part-classifier/tr...
6,1.2.826.0.1.3680043.8.498.10062189329714053601...,14,../input/unifesp-x-ray-body-part-classifier/tr...
7,1.2.826.0.1.3680043.8.498.10065930002825553435...,13 20,../input/unifesp-x-ray-body-part-classifier/tr...


In [9]:
train_df.shape

(1738, 3)

<a id="2"></a>

# <span><center style = "font-family:Verdana; font-size:36px; letter-spacing:2px; font-weight:bold; color:teal">2.&nbsp;&nbsp;CONVERTING IMAGE DATA TO JPG&nbsp;&nbsp;&nbsp;&nbsp;<a href='#toc'>&#10514;</a></center></span>

In [10]:
def read_dicom_image(path):
    dicom_file = dcmread(path)
    image = dicom_file.pixel_array
    image = image/image.max()
    return image

In [11]:
def save_image_jpg(path, save_path_name):
    image = read_dicom_image(path)*255
    plt.imsave(save_path_name, image)

In [12]:
os.listdir()

['__notebook_source__.ipynb', '.virtual_documents']

In [13]:
# shutil.rmtree('./train')

In [17]:
# Saving jpg images to working directory

# os.mkdir('./train')

for i in tqdm(range(500, 1000)):
    save_image_jpg(train_df.path[i], f'./train/{train_df.SOPInstanceUID[i]}.jpg')

100%|██████████| 500/500 [31:15<00:00,  3.75s/it]


In [1]:
file_names, file_paths = image_path_creation(config.TRAIN_JPG_PATH, '.jpg')
image_df = {'SOPInstanceUID': file_names, 'path_jpg':file_paths}
image_df = pd.DataFrame(image_df)

NameError: name 'image_path_creation' is not defined

In [16]:
os.listdir(config.TRAIN_JPG_PATH)

FileNotFoundError: [Errno 2] No such file or directory: '../input/unifesp-xray-jpg-dataset/train'

In [17]:
image_df#, train_df.SOPInstanceUID[0]

,SOPInstanceUID,path_jpg


In [ ]:
train_df_jpg=pd.merge(train_df, image_df, on = 'SOPInstanceUID')

In [ ]:
train_df_jpg.head()

In [ ]:
fig = plt.figure(figsize=(25,15))
for i in range(15):
    ax = fig.add_subplot(3,5, i+1)
#     image = read_dicom_image(train_df.path_jpg[i*5])
    img = np.array(Image.open(train_df_jpg.path_jpg[i*5]))
    plt.imshow(img, cmap = 'gray')
    ax.set_title(train_df.Target[i*5])


In [ ]:
img = img/255

In [ ]:
cv2.ConvertColor(img, )